In [17]:
import re
import pandas as pd


def parse_txt_to_dict_list(file_path):
    with open(file_path, 'r') as file:
        data = file.read()
    
    # Separando as sessões pelo delimitador de linha
    sessions = data.split("#######################################")
    dict_list = []

    for session in sessions:
        session = session.strip()
        if not session:
            continue

        # Extrai os parâmetros gerais
        ema_short = int(re.search(r'Ema short = (\d+)', session).group(1))
        donchian_window = int(re.search(r'Donchian Window = (\d+)', session).group(1))
        donchian_window_prev = int(re.search(r'donchian_window_prev = (\d+)', session).group(1))

        # Encontra os pares dentro da sessão
        pairs = re.findall(r'Pair ---------> (.*?)\nSL => (\d+) \| TP => (\d+)\n\nResult: ([\d\.\-]+)\nLen loss: *(\d+)\nLen Open: *(\d+)\nLen Close: *(\d+)\nLen Pos *(\d+)\nLen Neg *(\d+)\nRes pos *([\d\.\-]+)\nRes neg *([\d\.\-]+)\nRel len pos neg *([\d\.\-]+)\nRel len neg pos *([\d\.\-]+)\nRel pos neg *([\d\.\-]+)\nRel neg pos *([\d\.\-]+)', session)
        
        for pair in pairs:
            pair_dict = {
                "Ema_short": ema_short,
                "Donchian_Window": donchian_window,
                "Donchian_Window_Prev": donchian_window_prev,
                "Pair": pair[0],
                "SL": int(pair[1]),
                "TP": int(pair[2]),
                "Result": float(pair[3]),
                "Len_Loss": int(pair[4]),
                "Len_Open": int(pair[5]),
                "Len_Close": int(pair[6]),
                "Len_Pos": int(pair[7]),
                "Len_Neg": int(pair[8]),
                "Res_Pos": float(pair[9]),
                "Res_Neg": float(pair[10]),
                "Rel_Len_Pos_Neg": float(pair[11]),
                "Rel_Len_Neg_Pos": float(pair[12]),
                "Rel_Pos_Neg": float(pair[13]),
                "Rel_Neg_Pos": float(pair[14])
            }
            dict_list.append(pair_dict)

    return dict_list


In [18]:

# File paths for the two files
file_path = 'results-donchian-high-low-75-25-mid-M5-3.txt'
parsed_data_from_files = parse_txt_to_dict_list(file_path)

# Exibindo o resultado
# for data in parsed_data_from_files:
#     print(data)


In [19]:
import pandas as pd

# Criando o DataFrame

df = pd.DataFrame(parsed_data_from_files)


# Supondo que já tenha os dados carregados no DataFrame 'df'
# Agrupamento por 'Pair' e análise dos resultados negativos e positivos
grouped_pairs = df.groupby("Pair").agg({
    "Result": ["sum", "mean"],
    "Rel_Pos_Neg": "mean",
    "Rel_Neg_Pos": "mean"
}).reset_index()

# Renomeando colunas para facilitar a visualização
grouped_pairs.columns = ["Pair", "Total Result", "Mean Result", "Mean Rel_Pos_Neg", "Mean Rel_Neg_Pos"]

# Separando pares com resultado positivo e negativo
positive_results = df[df["Result"] > 0]
negative_results = df[df["Result"] < 0]

# Rankings
ranking_result = df.sort_values(by="Result", ascending=False)
ranking_rel_pos_neg = df.sort_values(by="Rel_Pos_Neg", ascending=False)
ranking_rel_neg_pos = df.sort_values(by="Rel_Neg_Pos", ascending=False)

# Exibindo resultados
print("Grouped Pairs Analysis:")
grouped_pairs



Grouped Pairs Analysis:


,Pair,Total Result,Mean Result,Mean Rel_Pos_Neg,Mean Rel_Neg_Pos
0,EURGBP,-127914.90,-4568.389286,0.463314,0.536686
1,EURJPY,-60281.80,-2232.659259,0.498915,0.501085
2,EURUSD,-31237.90,-1201.457692,0.500619,0.499381
3,GBPJPY,-825.00,-30.555556,0.502416,0.497584
4,GBPUSD,-138037.60,-5112.503704,0.473263,0.526737
5,USDJPY,29715.65,1100.579630,0.513558,0.486442
6,XAUUSD,1515253.00,58278.961538,0.517360,0.482640


In [20]:
print("\nRanking by Result:")
ranking_result



Ranking by Result:


,Ema_short,Donchian_Window,Donchian_Window_Prev,Pair,SL,TP,Result,Len_Loss,Len_Open,Len_Close,Len_Pos,Len_Neg,Res_Pos,Res_Neg,Rel_Len_Pos_Neg,Rel_Len_Neg_Pos,Rel_Pos_Neg,Rel_Neg_Pos
97,20,500,100,XAUUSD,50000,50000,189836.00,106,2,1666,1326,340,1264597.0,1074761.00,0.795918,0.204082,0.540574,0.459426
151,50,500,100,XAUUSD,50000,50000,179784.00,90,2,1399,1101,298,1179703.0,999919.00,0.786991,0.213009,0.541242,0.458758
172,50,2500,100,XAUUSD,50000,50000,170245.00,41,2,581,500,81,701677.0,531432.00,0.860585,0.139415,0.569031,0.430969
104,20,500,250,XAUUSD,50000,50000,158547.00,83,2,1499,1159,340,1205270.0,1046723.00,0.773182,0.226818,0.535201,0.464799
90,20,500,10,XAUUSD,50000,50000,153217.00,206,2,2801,2422,379,1500518.0,1347301.00,0.864691,0.135309,0.526901,0.473099
118,20,2500,100,XAUUSD,50000,50000,146844.00,52,1,707,605,102,770121.0,623277.00,0.855728,0.144272,0.552693,0.447307
132,20,5000,10,XAUUSD,50000,50000,137615.00,36,1,733,677,56,629752.0,492137.00,0.923602,0.076398,0.561332,0.438668
125,20,2500,250,XAUUSD,50000,50000,131325.00,48,1,632,529,103,752100.0,620775.00,0.837025,0.162975,0.547828,0.452172
179,50,2500,250,XAUUSD,50000,50000,128897.00,42,1,509,422,87,708053.0,579156.00,0.829077,0.170923,0.550068,0.449932
111,20,2500,10,XAUUSD,50000,50000,125687.00,67,1,1125,1019,106,832356.0,706669.00,0.905778,0.094222,0.540833,0.459167


In [21]:
print("\nRanking by Rel_Pos_Neg:")
ranking_rel_pos_neg



Ranking by Rel_Pos_Neg:


,Ema_short,Donchian_Window,Donchian_Window_Prev,Pair,SL,TP,Result,Len_Loss,Len_Open,Len_Close,Len_Pos,Len_Neg,Res_Pos,Res_Neg,Rel_Len_Pos_Neg,Rel_Len_Neg_Pos,Rel_Pos_Neg,Rel_Neg_Pos
139,50,500,10,EURJPY,50000,50000,23129.30,12,7,298,226,72,43438.2,20308.90,0.758389,0.241611,0.681415,0.318585
164,50,2500,10,EURUSD,50000,50000,10530.60,4,2,116,84,32,20368.7,9838.10,0.724138,0.275862,0.674308,0.325692
185,50,5000,10,EURUSD,50000,50000,2552.00,3,4,45,33,12,6219.3,3667.30,0.733333,0.266667,0.629064,0.370936
71,5,10000,100,EURJPY,50000,50000,10168.30,21,2,596,539,57,27182.2,17013.90,0.904362,0.095638,0.615036,0.384964
64,5,10000,10,EURJPY,50000,50000,8266.00,24,3,792,746,46,24383.2,16117.20,0.941919,0.058081,0.602048,0.397952
140,50,500,10,GBPJPY,50000,50000,23426.60,26,8,424,309,115,69517.3,46090.70,0.728774,0.271226,0.601319,0.398681
73,5,10000,100,GBPUSD,50000,50000,9326.40,43,3,690,625,65,32286.2,22959.80,0.905797,0.094203,0.584408,0.415592
123,20,2500,250,USDJPY,50000,50000,10659.30,34,2,809,680,129,38243.5,27584.20,0.840544,0.159456,0.580964,0.419036
80,5,10000,250,GBPUSD,50000,50000,8691.90,39,3,622,565,57,32145.8,23453.90,0.908360,0.091640,0.578165,0.421835
181,50,5000,10,EURJPY,50000,50000,3536.10,2,1,72,55,17,13855.8,10319.70,0.763889,0.236111,0.573134,0.426866


In [22]:
print("\nRanking by Rel_Neg_Pos:")
ranking_rel_neg_pos


Ranking by Rel_Neg_Pos:


,Ema_short,Donchian_Window,Donchian_Window_Prev,Pair,SL,TP,Result,Len_Loss,Len_Open,Len_Close,Len_Pos,Len_Neg,Res_Pos,Res_Neg,Rel_Len_Pos_Neg,Rel_Len_Neg_Pos,Rel_Pos_Neg,Rel_Neg_Pos
162,50,2500,10,GBPUSD,50000,50000,-40972.60,29,2,120,68,52,18369.7,59342.30,0.566667,0.433333,0.236382,0.763618
143,50,500,10,EURUSD,50000,50000,-31132.20,91,5,394,205,189,20880.5,52012.70,0.520305,0.479695,0.286453,0.713547
183,50,5000,10,GBPUSD,50000,50000,-13811.60,16,2,54,26,28,10131.7,23943.30,0.481481,0.518519,0.297335,0.702665
141,50,500,10,GBPUSD,50000,50000,-28555.10,54,6,370,235,135,38566.5,67121.60,0.635135,0.364865,0.364909,0.635091
88,20,500,10,USDJPY,50000,50000,-45758.95,306,2,3891,3260,631,84879.5,130638.45,0.837831,0.162169,0.393840,0.606160
83,5,10000,250,XAUUSD,50000,50000,-267158.00,32,1,470,417,53,541550.0,808708.00,0.887234,0.112766,0.401071,0.598929
184,50,5000,10,USDJPY,50000,50000,-6681.80,11,0,83,56,27,13910.0,20591.80,0.674699,0.325301,0.403167,0.596833
36,5,2500,250,EURJPY,50000,50000,-16599.90,135,4,1237,950,287,48697.8,65297.70,0.767987,0.232013,0.427191,0.572809
49,5,5000,100,EURGBP,50000,50000,-6914.30,144,7,953,758,195,21597.7,28512.00,0.795383,0.204617,0.431008,0.568992
42,5,5000,10,EURGBP,50000,50000,-6264.50,132,5,1122,958,164,20181.8,26446.30,0.853832,0.146168,0.432825,0.567175


In [23]:
import pandas as pd

# Configuração para exibir todas as linhas e colunas
pd.set_option('display.max_rows', None)  # Exibe todas as linhas
pd.set_option('display.max_columns', None)  # Exibe todas as colunas

# Suponha que o DataFrame 'df' já contenha os dados

# Ordenando o DataFrame por 'Pair' e 'Result' para cada par
ranked_result_per_pair = df.sort_values(by=["Pair", "Result"], ascending=[True, False]).reset_index(drop=True)

# Ordenando o DataFrame por 'Pair' e 'Rel_Pos_Neg' para cada par
ranked_rel_pos_neg_per_pair = df.sort_values(by=["Pair", "Rel_Pos_Neg"], ascending=[True, False]).reset_index(drop=True)

# Exibindo os rankings de todos os valores de 'Result' e 'Rel_Pos_Neg' para cada par
print("Ranking of Result for Each Pair:")
ranked_result_per_pair



Ranking of Result for Each Pair:


,Ema_short,Donchian_Window,Donchian_Window_Prev,Pair,SL,TP,Result,Len_Loss,Len_Open,Len_Close,Len_Pos,Len_Neg,Res_Pos,Res_Neg,Rel_Len_Pos_Neg,Rel_Len_Neg_Pos,Rel_Pos_Neg,Rel_Neg_Pos
0,50,5000,10,EURGBP,50000,50000,1680.10,14,2,81,57,24,10275.9,8595.80,0.703704,0.296296,0.544514,0.455486
1,50,2500,250,EURGBP,50000,50000,-167.50,82,1,656,515,141,17335.2,17502.70,0.785061,0.214939,0.497596,0.502404
2,50,2500,100,EURGBP,50000,50000,-367.50,79,1,667,535,132,17271.4,17638.90,0.802099,0.197901,0.494737,0.505263
3,5,10000,250,EURGBP,50000,50000,-1348.10,79,11,623,519,104,16473.1,17821.20,0.833066,0.166934,0.480345,0.519655
4,5,10000,100,EURGBP,50000,50000,-1446.70,69,10,606,513,93,15342.1,16788.80,0.846535,0.153465,0.477487,0.522513
5,20,2500,100,EURGBP,50000,50000,-1937.60,120,2,893,693,200,20805.9,22743.50,0.776036,0.223964,0.477754,0.522246
6,50,2500,10,EURGBP,50000,50000,-2070.70,28,2,108,70,38,11092.4,13163.10,0.648148,0.351852,0.457315,0.542685
7,50,5000,100,EURGBP,50000,50000,-2336.00,59,4,436,353,83,12233.6,14569.60,0.809633,0.190367,0.456423,0.543577
8,20,2500,10,EURGBP,50000,50000,-2651.30,153,3,1178,969,209,23094.2,25745.50,0.822581,0.177419,0.472857,0.527143
9,20,2500,250,EURGBP,50000,50000,-3047.00,117,2,873,654,219,20829.2,23876.20,0.749141,0.250859,0.465921,0.534079


In [24]:
print("\nRanking of Rel_Pos_Neg for Each Pair:")

ranked_rel_pos_neg_per_pair


Ranking of Rel_Pos_Neg for Each Pair:


,Ema_short,Donchian_Window,Donchian_Window_Prev,Pair,SL,TP,Result,Len_Loss,Len_Open,Len_Close,Len_Pos,Len_Neg,Res_Pos,Res_Neg,Rel_Len_Pos_Neg,Rel_Len_Neg_Pos,Rel_Pos_Neg,Rel_Neg_Pos
0,50,5000,10,EURGBP,50000,50000,1680.10,14,2,81,57,24,10275.9,8595.80,0.703704,0.296296,0.544514,0.455486
1,50,2500,250,EURGBP,50000,50000,-167.50,82,1,656,515,141,17335.2,17502.70,0.785061,0.214939,0.497596,0.502404
2,50,2500,100,EURGBP,50000,50000,-367.50,79,1,667,535,132,17271.4,17638.90,0.802099,0.197901,0.494737,0.505263
3,5,10000,250,EURGBP,50000,50000,-1348.10,79,11,623,519,104,16473.1,17821.20,0.833066,0.166934,0.480345,0.519655
4,20,500,250,EURGBP,50000,50000,-3131.60,214,3,2055,1426,629,38119.0,41250.60,0.693917,0.306083,0.480272,0.519728
5,20,500,100,EURGBP,50000,50000,-3522.40,283,3,2337,1679,658,40074.8,43597.20,0.718442,0.281558,0.478951,0.521049
6,20,2500,100,EURGBP,50000,50000,-1937.60,120,2,893,693,200,20805.9,22743.50,0.776036,0.223964,0.477754,0.522246
7,5,10000,100,EURGBP,50000,50000,-1446.70,69,10,606,513,93,15342.1,16788.80,0.846535,0.153465,0.477487,0.522513
8,20,2500,10,EURGBP,50000,50000,-2651.30,153,3,1178,969,209,23094.2,25745.50,0.822581,0.177419,0.472857,0.527143
9,50,500,100,EURGBP,50000,50000,-4574.80,246,1,1796,1272,524,34333.5,38908.30,0.708241,0.291759,0.468769,0.531231
